In [1]:
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.3/901.3 kB 29.8 MB/s eta 0:00:00


In [2]:
import cv2
import time  # Import to measure time
from ultralytics import YOLO, SAM

# Configuration
SCALE_FACTOR = 1
START_IDX = 10
END_IDX = 450
SOURCE_VIDEO = '/content/drive/MyDrive/Techfest/basketball-1 (2).mp4'
OUTPUT_VIDEO = '/content/huppa.mp4'

# Load custom YOLO weight and SAM models
model_yolo = YOLO('/content/drive/MyDrive/Techfest/weights/best.pt')
model_sam = SAM("sam2_b.pt")

# Extract frames and process video
cap = cv2.VideoCapture(SOURCE_VIDEO)
if not cap.isOpened():
    raise ValueError("Error: Could not open video.")

# Get input video FPS for output video
input_fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
output_frames = []

# FPS calculation variables
frame_processing_times = []

for idx in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break

    if START_IDX <= idx < END_IDX:
        frame = cv2.resize(frame, None, fx=SCALE_FACTOR, fy=SCALE_FACTOR)

        # Start time for FPS calculation
        start_time = time.time()

        # Run YOLO detection
        results = model_yolo.track(frame, persist=True)
        bounding_boxes = results[0].boxes.xyxy.cpu().numpy()
        confidences = results[0].boxes.conf.cpu().numpy()

        # Use YOLO bounding boxes with SAM
        if len(bounding_boxes) > 0:
            sam_results = model_sam(frame, bboxes=bounding_boxes, labels=[1] * len(bounding_boxes))

            # Annotate frame
            annotated_frame = frame.copy()
            masks = getattr(sam_results, "masks", [])

            for i, (box, confidence) in enumerate(zip(bounding_boxes, confidences)):
                x1, y1, x2, y2 = map(int, box)

                # Draw bounding box with confidence
                color = (0, 255, 0) if confidence > 0.7 else (0, 165, 255)
                label = f"Obj {i+1}: {confidence:.2f}"
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(annotated_frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Add mask overlay directly from SAM
                if masks and len(masks) > i:
                    mask_overlay = masks[i].astype(np.uint8) * 255
                    color_mask = cv2.applyColorMap(mask_overlay, cv2.COLORMAP_JET)
                    annotated_frame = cv2.addWeighted(annotated_frame, 0.7, color_mask, 0.3, 0)

            output_frames.append(annotated_frame)

        # End time for FPS calculation
        end_time = time.time()
        frame_processing_times.append(end_time - start_time)

cap.release()

# Calculate average FPS of solution
if frame_processing_times:
    solution_fps = len(frame_processing_times) / sum(frame_processing_times)
    print(f"Solution FPS (Processing Speed): {solution_fps:.2f}")
else:
    print("No frames were processed. Solution FPS could not be calculated.")

# Save annotated video with input video FPS
if output_frames:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, input_fps, (frame_width, frame_height))
    for frame in output_frames:
        out.write(frame)
    out.release()
    print(f"Output video saved as {OUTPUT_VIDEO}")
else:
    print("No frames were processed. Output video not created.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 154M/154M [00:01<00:00, 107MB/s]


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.7s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 640x1088 (no detections), 58.2ms
Speed: 14.9ms preprocess, 58.2ms inference, 109.0ms postprocess per image at shape (1, 3, 640, 1088)

0: 640x1088 (no detections), 56.1ms
Speed: 8.4ms preprocess, 56.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 1088)

0: 640x1088 (no detections), 55.9ms
Speed: 7.4ms preprocess, 55.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 1088)

0: 640x1088 (no detections), 43.2ms
Speed: 7.3ms preprocess, 43.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 1088)

0: 640x1088 (no detections), 42.8ms
Speed: 7.2ms preprocess, 42.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 1088